In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
import cv2
from PIL import Image
from skimage import exposure
from skimage.transform import resize
import scipy
import time
from statistics import mean
import tensorflow as tf
import keras as ke
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
Tabela_treino = pd.read_csv('/kaggle/input/spr-x-ray-age/train_age.csv',sep = ',',engine = 'python')
display(Tabela_treino)

,imageId,age
0,0,89.0
1,1,72.0
2,2,25.0
3,3,68.0
4,4,37.0
...,...,...
10697,10697,79.0
10698,10698,67.0
10699,10699,66.0
10700,10700,78.0


In [6]:
nlin = np.shape(Tabela_treino)[0]   #numero de linhas da minha tabela do excel
hhh = []
tamy = []

for i in tqdm(range(0,nlin)):
    imageID = Tabela_treino['imageId'].iloc[i]      #pega o valor do imageID de cada linha 
    file_path = ("/kaggle/input/spr-x-ray-age/kaggle/kaggle/train/"  + str(imageID).zfill(6) + ".png")
    img = cv2.imread(file_path) #aplica o numero do imageID pra buscar a imagem no diretório com o mesmo numero
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.merge((img,img,img))
    img = cv2.resize(img, (200,200), interpolation = cv2.INTER_AREA)
    img = np.array(img)/255
    hhh.append(img)
    tamy.append(Tabela_treino['age'].iloc[i])
tamy = np.array(tamy)
    

100%|██████████| 10702/10702 [09:21<00:00, 19.07it/s]


In [9]:
X_treino, X_val, Y_treino, Y_val = train_test_split(hhh, tamy, test_size=0.2, random_state=50,shuffle=True)

In [7]:
tamy = 0 #list(Y_all).clear()
hhh.clear()

print(tamy, hhh)
print(np.shape(X_treino), np.shape(X_val), np.shape(Y_treino), np.shape(Y_val))

0 []
(8561, 200, 200, 3) (2141, 200, 200, 3) (8561,) (2141,)


# Sem Kfold

In [ ]:
from keras.applications import EfficientNetB2
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

start_time = time.time()

np.random.RandomState(1) # Fixar o pseudo-random generator do numpy
tf.random.set_seed(1) # Fixar o pseudo-random generator do tensorflow
# criando a base da EfficientNet pré-treinada
base_model = EfficientNetB2(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

# adicionando camadas de classificação no topo da base_model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1)(x)

# definindo o modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# congelando as camadas da base_model
for layer in base_model.layers:
    layer.trainable = False

opt = ke.optimizers.Adam( learning_rate=0.01,
beta_1=0.9,
beta_2=0.999,
epsilon=1e-07,
amsgrad=False)
#callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model.compile(optimizer= opt, loss='mse',metrics=['mae'])  # Compilando o modelo

history = model.fit(np.array(X_treino), Y_treino,batch_size = 100, epochs = 50, validation_data = (np.array(X_val), Y_val))
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(history.history["loss"], label="treino")
plt.plot(history.history["val_loss"], label="validacao")
plt.xlabel("epocas")
plt.ylabel("Loss")
plt.ylim( bottom = 0, top = 0.5)
plt.legend(loc='upper right');
plt.subplot(1,2,2)
plt.plot(history.history["accuracy"], label="treino")
plt.plot(history.history["val_accuracy"], label="validacao")
plt.xlabel("epocas")
plt.ylabel("Acuracia")
plt.ylim( bottom = 0.9, top = 1)
plt.legend(loc='lower right');

In [ ]:
#salvar o modelo do melhor fold
model.save('modelclass(female)sem_kfold_tranfer_learning.h5')

# Com Kfold

In [ ]:
from keras.applications import EfficientNetB2
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

start_time = time.time()

#iniciando as listas
mae_por_fold = []
loss_por_fold = []
all_models=[]
all_loss=[]
all_mae=[]
all_vloss=[]
all_vmae=[]

#Dados de entrada
inputs = np.array(hhh)
targets = tamy

# escolher o numero de folds
num_folds = 5

# Validação cruzada
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_no = 1
for train, test in kfold.split(inputs, targets):

    # criando a base da ResNet50 pré-treinada
    base_model = EfficientNetB2(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

    # adicionando camadas de classificação no topo da base_model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(1)(x)

    # definindo o modelo final
    model = Model(inputs=base_model.input, outputs=predictions)

    # congelando as camadas da base_model
    for layer in base_model.layers:
        layer.trainable = False

    opt = ke.optimizers.Adam( learning_rate=0.01,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False)
    #callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
    model.compile(optimizer= opt, loss='mse',metrics=['mae'])  # Compilando o modelo
    

    print('------------------------------------------------------------------------')
    print(f'Treinamento para o fold {fold_no} ...')
    
    # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=100,
              #callbacks=[callback],
              epochs=100,
              validation_data=(inputs[test], targets[test]))

  
    mse, mae =  model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score para o fold {fold_no}: {model.metrics_names[0]} de {mse}; {model.metrics_names[1]} de {mae}')
    mae_por_fold.append(mae) # aqui estamos criando uma lista com as accs dos 5 folds
    loss_por_fold.append(mse)    # aqui estamos criando uma lista com as loss dos 5 folds
    
   # Increase fold number
    fold_no = fold_no + 1
    
    
    all_models.append(model)
    
    all_vloss.append(history.history["val_loss"])
    all_vmae.append(history.history["val_mae"])
    all_loss.append(history.history["loss"])
    all_mae.append(history.history["mae"])
    
print("--- %s seconds ---" % (time.time() - start_time))

31790344/31790344 [==============================] - 2s 0us/step
------------------------------------------------------------------------
Treinamento para o fold 1 ...


In [ ]:
#salvar o modelo do melhor fold
model.save('model_kfold_tranfer_learning.h5')

In [ ]:
import pickle

# Salvar todas as variáveis em um arquivo pickle
with open("all_variables.pkl", "wb") as f:
    pickle.dump([all_models, all_vloss, all_vacc, all_loss, all_acc, acc_por_fold,loss_por_fold], f)

# # Carregar as variáveis a partir do arquivo pickle
# with open("all_variables.pkl", "rb") as f:
#     all_models, all_vloss, all_vacc, all_loss, all_acc, acc_por_fold,loss_por_fold = pickle.load(f)

In [ ]:
# Dados de Saida
print('------------------------------------------------------------------------')
print('Scores para cada um dos folds:')

for i in range(0, len(acc_por_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_por_fold[i]} - acc: {acc_por_fold[i]}')
    
print('------------------------------------------------------------------------')
print('Média dos scores para todos os folds:')
print(f'> acc: {np.mean(acc_por_fold)} (+- {np.std(acc_por_fold)})')
print(f'> Loss: {np.mean(loss_por_fold)}(+- {np.std(acc_por_fold)})')
print('------------------------------------------------------------------------')
print(f'> Tamanho do conjunto de treinamento e conjunto de teste por fold: {(len(train), len(test))}')



fig, axs = plt.subplots(2, 2, figsize=(15,10))
for i in range(num_folds):
    axs[0, 0].plot(all_loss[i], label="fold"+str(i+1))
    axs[0, 0].set_title('loss (treino) por fold')
    axs[0, 0].legend(loc='upper right')
    axs[0, 0].set_xlabel('Epochs')
    axs[0, 0].set_ylabel('Loss')
    axs[0, 0].grid()

    axs[0, 1].plot(all_vloss[i], label="fold"+str(i+1))
    axs[0, 1].set_title('loss (validacao) por fold')
    axs[0, 1].legend(loc='upper right')
    axs[0, 1].set_xlabel('Epochs')
    axs[0, 1].set_ylabel('Validação Loss')
    axs[0, 1].grid()

    axs[1, 0].plot(all_acc[i], label="fold"+str(i+1))
    axs[1, 0].set_title('acc (treino) por fold')
    axs[1, 0].legend(loc='lower right')
    axs[1, 0].set_xlabel('Epochs')
    axs[1, 0].set_ylabel('acc')
    axs[1, 0].grid()

    axs[1, 1].plot(all_vacc[i], label="fold"+str(i+1))
    axs[1, 1].set_title('acc (validacao) por fold')
    axs[1, 1].legend(loc='lower right')
    axs[1, 1].set_xlabel('Epochs')
    axs[1, 1].set_ylabel('Validação acc')
    axs[1, 1].grid()

# plt.savefig('img0.png', bbox_inches='tight')

In [ ]:
def smooth_curve(points, factor=0.7):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

smooth_vacc_history = []
smooth_acc_history = []
smooth_vloss_history = []
smooth_loss_history = []

for i in range(0,num_folds):
    Vacc = smooth_curve(all_vacc[i][50:])
    acc = smooth_curve(all_acc[i][50:])
    VLoss = smooth_curve(all_vloss[i][50:])
    Loss = smooth_curve(all_loss[i][50:])
    
    smooth_vacc_history.append(Vacc)
    smooth_acc_history.append(acc)
    smooth_vloss_history.append(VLoss)
    smooth_loss_history.append(Loss)

In [ ]:
print('------------------------------------------------------------------------')
print('Média dos scores para todos os folds:')
print(f'> acc: {np.mean(acc_por_fold)} (+- {np.std(acc_por_fold)})')
#print(f'> Loss: {np.mean(loss_por_fold)}(+- {np.std(acc_por_fold)})')
print('------------------------------------------------------------------------')
print(f'> Tamanho do conjunto de treinamento e validação por fold: {(len(train), len(test))}')


fig, axs = plt.subplots(2, 2, figsize=(15,10))
for i in range(num_folds):
    axs[0, 0].plot(smooth_loss_history[i],label="fold"+str(i+1))
    axs[0, 0].set_title('loss (treino) por fold')
    axs[0, 0].legend(loc='upper right')
    axs[0, 0].set_xlabel('Epochs')
    axs[0, 0].set_ylabel('Loss')
    axs[0, 0].grid()
    axs[0, 1].plot(smooth_vloss_history[i],label="fold"+str(i+1))
    axs[0, 1].set_title('loss (validacao) por fold')
    axs[0, 1].legend(loc='upper right')
    axs[0, 1].set_xlabel('Epochs')
    axs[0, 1].set_ylabel('Validação Loss')
    axs[0, 1].grid()
    axs[1, 0].plot(smooth_acc_history[i],label="fold"+str(i+1))
    axs[1, 0].set_title('acc (dados de treino) por fold')
    axs[1, 0].legend(loc='lower right')
    axs[1, 0].set_xlabel('Epochs')
    axs[1, 0].set_ylabel('acc')
    axs[1, 0].grid()
    axs[1, 1].plot(smooth_vacc_history[i],label="fold"+str(i+1))
    axs[1, 1].set_title('acc (dados de validacao) por fold')
    axs[1, 1].legend(loc='lower right')
    axs[1, 1].set_xlabel('Epochs')
    axs[1, 1].set_ylabel('Validação acc')
    axs[1, 1].grid()      
# plt.savefig('img1.png', bbox_inches='tight');